### Agents
Agents will provide a way for us to add additional sources of information that will build on our RAG approach.  We may need to execute code or search different information sources to build a complex context to answer user queries.

### Tools

In [1]:
# %pip install langchain_community -q
# %pip install tavily-python -q
# %pip install python-dotenv -q
# %pip install langchainhub -q
#%pip install wikibase-rest-api-client mediawikiapi -q
#%pip install yfinance
#%pip install -U langchain-aws
#%pip install --upgrade langchain-aws
#%pip install --upgrade langchain-core


In [ ]:
import os
import boto3

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.chat_models import BedrockChat
from langchain_core.messages.content_blocks import is_data_content_block
from langchain_core.messages import AIMessage, HumanMessage
from langchain import hub
from dotenv import load_dotenv

load_dotenv('/Users/marvinlee/Documents/vs_code/gsb_570/.env')
# Create the AWS client for the Bedrock runtime with boto3
aws_client = boto3.client(service_name="bedrock-runtime")

USER_AGENT environment variable not set, consider setting it to identify your requests.


### Agent Setup
We need three components to make up our agent
1) Tools we plan to use to execute respond to input
2) LLM we plan to use as our logic executor
3) Agent type we plan to use for this use case

#### Tavily Search
Tavily's Search API is a search engine built specifically for AI agents (LLMs) that LangChain supports as a library import.

*Note: This tool requires an API Key that will be loaded in your .env file as follows*.  

>TAVILY_API_KEY=tvly-XXXXXXXXXXXXXXXXXXX

To exectute this notebook [visit](https://app.tavily.com/) for an API key.


In [2]:
test = os.getenv('TAVILY_API_KEY')
print("API=",test)

API= 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


In [3]:
# 1 Start with a simple web search tool:
tools = [TavilySearchResults(max_results=1, TAVILY_API_KEY = os.getenv('TAVILY_API_KEY'))]

# 2 LLM, Let's use Claude's Haiku
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

# Grab our LLM of choice
model_kwargs =  { 
    "max_tokens": 2048,
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 0.9,
    "stop_sequences": ["\n\nHuman"],
}
llm = BedrockChat(
    client=aws_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

# Get the prompt to use with our agent
# The hub provides sample templates for each agent type
prompt = hub.pull("hwchase17/structured-chat-agent")


#print("Tools:", tools)
#print("Prompt", prompt)



/var/folders/m1/tx6ck0f962jd1hq4ch5phg_80000gn/T/ipykernel_4056/2159499168.py:15: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.0.34 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import ChatBedrock``.
  llm = BedrockChat(
/Users/marvinlee/Documents/vs_code/gsb_570/gsb570env/lib/python3.12/site-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [4]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.System

#### Our logic prompt
Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```

{{

  "action": $TOOL_NAME,

  "action_input": $INPUT

}}

```

Follow this format:

Question: input question to answer

Thought: consider previous and subsequent steps

Action:

```

$JSON_BLOB

```

Observation: action result

... (repeat Thought/Action/Observation N times)

Thought: I know what to respond

Action:

```

{{

  "action": "Final Answer",

  "action_input": "Final response to human"

}}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation

In [5]:
# 3 Construct the create_structured_chat_agent agent
# https://python.langchain.com/v0.1/docs/modules/agents/agent_types/
agent = create_structured_chat_agent(llm, tools, prompt)

# run the agent
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

agent_executor.invoke({"input": "What happened to the economy historically when tariffs were added to US imports?"})



> Entering new AgentExecutor chain...
Okay, let's start by searching for relevant information on the economic impacts of tariffs on US imports:

Action:
```json
{
  "action": "tavily_search_results_json",
  "action_input": {
    "query": "economic impact of tariffs on US imports"
  }
}
```
[{'title': 'Tariffs: Estimating the Economic Impact of the 2025 Measures and ...', 'url': 'https://www.richmondfed.org/publications/research/economic_brief/2025/eb_25-12', 'content': "Although these tariffs provided some targeted economic benefits by increasing employment in protected sectors, they ultimately produced a net loss to the U.S. economy. A 2019 working paper found that tariffs generated approximately $51 billion (about 0.27 percent of GDP) in losses for consumers and firms reliant on imported goods, though factoring in job gains within protected industries reduced the net loss to about $7.2 billion, or roughly 0.04 percent of GDP.7\n7 [...] Between 2018 and 2019, the U.S. imposed tariff

{'input': 'What happened to the economy historically when tariffs were added to US imports?',
 'output': 'Historically, when tariffs were added to US imports, it had a negative overall economic impact despite some targeted benefits for protected industries. Key impacts included:\n\n- Higher costs for US businesses and consumers that relied on imported goods/inputs, leading to reduced economic output and job losses in import-dependent industries. One study estimated a net loss of $7.2 billion or 0.04% of GDP from the 2018-2019 tariffs on China.\n\n- Disruptions to global supply chains as companies had to shift sourcing away from tariffed imports. This contributed to declines in manufacturing employment and heightened investment uncertainty.\n\n- While tariffs boosted employment in the specific protected domestic industries, the net effect was an estimated 1.8-2.6% decline in overall US employment (220,000-330,000 jobs lost) due to higher costs across many sectors.\n\n- Retaliatory tarif

In [8]:
response = agent_executor.invoke({"input": "Hi my name is Cybernewbee"})
last_input = response['input']
last_output = response['output']



> Entering new AgentExecutor chain...
```json
{
  "action": "tavily_search_results_json",
  "action_input": {
    "query": "Cybernewbee"
  }
}
```
[{'title': 'cyberNewbie (@tosinmabinuori) / X', 'url': 'https://twitter.com/tosinmabinuori', 'content': 'Cyber security | SALES AND SERVICE OF HOSPITAL EQUIPMENTS (X-RAY, MRI, CT, CR, ULTRASOUND | Healthcare | Biomedical Engineer | @morzitservices.', 'score': 0.40457857}]```json
{
  "action": "Final Answer",
  "action_input": "Based on the search results, it seems 'Cybernewbee' may refer to a Twitter account @tosinmabinuori who describes themselves as being involved in cybersecurity and biomedical engineering. However, without more context about your specific question, it's difficult for me to provide a more detailed response."
}
```

> Finished chain.


In [9]:
print(last_input)
print(last_output)

Hi my name is Cybernewbee
Based on the search results, it seems 'Cybernewbee' may refer to a Twitter account @tosinmabinuori who describes themselves as being involved in cybersecurity and biomedical engineering. However, without more context about your specific question, it's difficult for me to provide a more detailed response.


#### Can agents keep track of our conversation and use the context?
If we save the results, LangChain supports chat history

In [10]:
response = agent_executor.invoke({"input": "What's my name?"})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "I'm afraid I don't actually know your name. As an AI assistant without access to personal information about you, I can only respond based on the context provided in our conversation."
}
```

> Finished chain.


In [11]:
response = agent_executor.invoke({"input": "What's my name?", 
                                  "chat_history": [
                                        HumanMessage(content=last_input),
                                        AIMessage(content=last_output),
                                                  ],})




> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "Your name is Cybernewbee, based on the introduction you provided."
}
```

> Finished chain.


#### Wiki Data Tool
Let's add anther tool and observe how our Agent can use both tools to answer input questions

In [12]:
# New tool WikiData
wikidata = WikidataQueryRun(api_wrapper=WikidataAPIWrapper())
# Add WikiData and Search Results
tools = [wikidata, TavilySearchResults(max_results=1, TAVILY_API_KEY = os.getenv('TAVILY_API_KEY'))]
agent = create_structured_chat_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)
agent_executor.invoke({"input": "Who was the historical figure named Richard Feynman?"})



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "Wikidata",
  "action_input": {
    "query": "Richard Feynman"
  }
}
```
Result Q39246:
Label: Richard Feynman
Description: American theoretical physicist (1918–1988)
Aliases: Richard Phillips Feynman, Ofey, Feynman, Dick Feynman, Richard P. Feynman
instance of: human
country of citizenship: United States
occupation: physicist, quantum physicist, inventor, writer, university teacher, percussionist, theoretical physicist, science communicator, politician
sex or gender: male
date of birth: 1918-05-11
date of death: 1988-02-15
place of birth: Queens, Far Rockaway
educated at: Far Rockaway High School, Massachusetts Institute of Technology, Princeton University
employer: Cornell University, California Institute of Technology, Manhattan Project
place of death: Los Angeles, UCLA Ronald Reagan Medical Center
field of work: quantum electrodynamics, particle physics, physics
cause of death: kidney failure, abdominal cancer
no

{'input': 'Who was the historical figure named Richard Feynman?',
 'output': "Richard Feynman was an American theoretical physicist who made major contributions to the development of quantum mechanics and quantum electrodynamics. Some key facts about him:\n\n- He was born in 1918 in New York and died in 1988 in Los Angeles. \n- He won the Nobel Prize in Physics in 1965 for his work on quantum electrodynamics.\n- He worked on the Manhattan Project during World War II to develop the atomic bomb.\n- He taught at Cornell University and the California Institute of Technology.\n- He was known for his brilliant mind, eccentric personality, and ability to explain complex physics concepts in simple terms.\n- He wrote several popular books including 'Surely You're Joking, Mr. Feynman!' which described his life and adventures.\n- He made important contributions to areas like quantum computing, nanotechnology, and the physics of super fluidity.\n- He was an influential figure who inspired many stu

Now let's review our original question and see how the agent responds to the query.

In [13]:
agent_executor.invoke({"input": "What happened to the economy historically when tariffs were added to US imports?"})



> Entering new AgentExecutor chain...
Okay, let's break this down step-by-step:

Action: 
```json
{
  "action": "tavily_search_results_json",
  "action_input": "impact of tariffs on US economy"
}
```
[{'title': 'Trump Tariffs: The Economic Impact of the Trump Trade War', 'url': 'https://taxfoundation.org/research/all/federal/trump-tariffs-trade-war/', 'content': 'On a conventional basis, before incorporating the negative effects of tariffs on the US economy, we estimate all the tariffs together would increase US federal tax revenue by $2.1 trillion over the next decade. The April 2 “reciprocal” tariffs on the EU and countries excluding Canada, Mexico, and China increase tax revenue by $1.0 trillion, assuming the 90-day pause ends as scheduled. Notably, the tariffs on China, which for most imports are levied at 145 percent, raise comparatively small [...] Many economists have evaluated the consequences of the trade war tariffs on the American economy, with results suggesting the tarif

{'input': 'What happened to the economy historically when tariffs were added to US imports?',
 'output': 'Historically, when tariffs were added to US imports, it often had negative economic consequences. A major example is the Smoot-Hawley Tariff Act of 1930, which raised tariffs on thousands of imported goods. Many economists believe this worsened the Great Depression by raising consumer costs, provoking retaliation from US trade partners, and contracting global trade. More recently, the tariffs imposed by the Trump administration on imports from China and other countries in 2018-2019 are estimated to have raised costs for US businesses and consumers, lowered economic output and employment, and contributed to a slowdown in economic growth, according to analyses by economists and policy organizations.'}

#### Homework
Use the [langchain tool](https://python.langchain.com/api_reference/community/tools.html) list to find a useful tool other than Wikidata or Tavily to explore what it can do 

In [13]:
# Your tool here
# 1 Start with a simple web search tool:
#tools = -> FIND A GOOD TOOL


# 2 LLM, Let's use Claude's Haiku
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

# Grab our LLM of choice
model_kwargs =  { 
    "max_tokens": 2048,
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 0.9,
    "stop_sequences": ["\n\nHuman"],
}
llm = BedrockChat(
    client=aws_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

# Get the prompt to use with our agent
# The hub provides sample templates for each agent type
prompt = hub.pull("hwchase17/structured-chat-agent")

# 3 Construct the create_structured_chat_agent agent
# https://python.langchain.com/v0.1/docs/modules/agents/agent_types/
agent = create_structured_chat_agent(llm, tools, prompt)

# run the agent
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

agent_executor.invoke({"input": "What happened with the economy today?"})



> Entering new AgentExecutor chain...


/Users/marvinlee/Documents/vs_code/gsb_570/gsb570env/lib/python3.12/site-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": "economy today"
}
```
[{'title': "Economy | Today's latest from Al Jazeera", 'url': 'https://www.aljazeera.com/economy/', 'content': 'Trump trade war drives uncertainty across several industries. From carmakers to restaurant chains, companies face financial setbacks amid tariff fears.', 'score': 0.43266314}]```json
{
  "action": "tavily_search_results_json",
  "action_input": "us economy news today"
}
```
[{'title': 'US economy - Financial Times', 'url': 'https://www.ft.com/us-economy', 'content': 'News and headlines on the state of the US economy. The latest economic data, reports and updates from the US, including budgets and inflation forecasts.', 'score': 0.6601948}]```json
{
  "action": "Final Answer",
  "action_input": "Based on the search results, there does not appear to be any major breaking economic news or events in the US today. The results provide general information about the US economy from news so

{'input': 'What happened with the economy today?',
 'output': 'Based on the search results, there does not appear to be any major breaking economic news or events in the US today. The results provide general information about the US economy from news sources like the Financial Times and Al Jazeera, but no specific details on significant economic developments happening today.'}